#### Tradução de texto com DeepL e análise de sentimento e emoção em texto
 - Necessário conta na DeepL com assinatura gratuita para ter acesso a uma API KEY e 500mil caracteres/mês
 - O DeepL é uma opção para outros tradutores de texto com nível gratuitos e pagos, como Azure AI Translator e Google Translate
 - Necessário conta nos Cloud Services da IBM e da Azure para utilização dos recursos AZURE AI e IBM WATSON

In [1]:
from dotenv import load_dotenv
import os
import deepl # metodo 1
import requests # metodo 2
import pandas as pd

# Tratamento dos textos
import string # processing
import re # processing
from nltk.corpus import stopwords # processing
from nltk.tokenize import RegexpTokenizer # processing
import unidecode # processing

# Sentiment Analysis
from azure.core.credentials import AzureKeyCredential
from azure.ai.textanalytics import TextAnalyticsClient

# Emotion Analysis
from ibm_watson import NaturalLanguageUnderstandingV1
from ibm_cloud_sdk_core.authenticators import IAMAuthenticator
from ibm_watson.natural_language_understanding_v1 import Features, EmotionOptions

# Carrega as variáveis de ambiente do arquivo .env
load_dotenv()

api_key = os.getenv("DEEPL_KEY")
azure_key = os.getenv("AZURE_KEY")
azure_endpoint=os.getenv("AZURE_ENDPOINT")
ibm_key = os.getenv("IBM_KEY")

##### Utilizando a LIB da DeepL

In [2]:
# Test
translator = deepl.Translator(api_key)

result = translator.translate_text("Olá, mundo!", target_lang="EN-US")
print(result.text)  

Hello, world!


In [3]:
# Function para Dataframe
# Linguas suportadas: https://www.deepl.com/pt-BR/docs-api/glossaries/list-glossary-languages
def TranslateDeepL(text):
    result = result = translator.translate_text(text, \
        # delete a source_lang para detecção automática de idioma
        source_lang="PT", \
        target_lang="EN-US")
    
    return result

print(TranslateDeepL("Olá, mundo, testando!"))

Hello, world, testing!


##### Utilizando API Rest

In [4]:
def ReqDeepLTranslate(text):
    r = requests.post(
                url="https://api-free.deepl.com/v2/translate",
                data={
                    "source_lang": "PT", # deleta esta linha para detecção de idioma
                    "target_lang": "EN-US",
                    "auth_key": api_key,
                    "text": text,
                },
            )

    return r.json()["translations"][0]["text"]

print(ReqDeepLTranslate("A vida é uma só, aproveite!"))

Life is one, enjoy it!


##### Aplicando tradução a um dataframe

In [5]:
ds = pd.read_csv('data/ptbr-order_reviews.csv')
df = pd.DataFrame(ds.sample(200))
df = df[["score","title","opinion"]].dropna()

display(df)

,score,title,opinion
81316,5,Bom,Gostei comprarei outras vezes
47887,1,pessimo,produto com defeito.
16250,5,MUITO bom,Deu tudo certo
88016,5,Excelente,"Entrega rápida, produto de qualidade."
43520,5,super recomendo,produto muito útil
99108,5,Ótimo,Bom
23989,5,recomendo,produto entregue dentro do prazo e de acordo c...
46607,4,Recomendo,Produto bom.
78774,5,Super recomndo,O produto é lindo e de excelente qualidade
93284,4,Chegou antes do prazo,"Recomendo essa loja, o produto chegou antes do..."


In [6]:
# Tratamento de texto

def TextToLower(text):
    return text.lower()


def RemoveBlank(text):
    return text.strip()


def RemovePunctuation(text):
    punct = string.punctuation
    punct = punct.replace("'","")
    return text.translate(str.maketrans("","",punct))


def RemoveEmoji(text):
    emoji_pattern = re.compile("["
        u"\U0001F600-\U0001F64F" # emoticons
        u"\U0001F300-\U0001F5FF" # simbolos
        u"\U0001F680-\U0001F6FF" # transporte 
        u"\U0001F1E0-\U0001F1FF"
        u"\U00002702-\U000027B0" 
        u"\U000024C2-\U0001F251"
                    "]+", flags = re.UNICODE)
    
    return emoji_pattern.sub(r'', text) 


def RemoveHttp(text):
    http = "https?://\S+|www\.\S+"
    pattern = r"({})".format(http)
    return re.sub(pattern, "", text)


regexp = RegexpTokenizer("[\w']+")
def RemoveStopwords(text, stopwords):
    return " ".join(word for word in regexp.tokenize(text) if word.lower() not in stopwords)



In [7]:
stopwords_en = set(stopwords.words('english')) # opcional - pode ser utilizando 'english' após tradução do texto, ou 'portuguese' antes da tradução

def TextNormalizer(text):
    text = unidecode.unidecode(text)
    text = re.sub('\n','', text)
    text = RemoveHttp(text)
    text = RemoveEmoji(text)
    text = RemovePunctuation(text)
    text = RemoveStopwords(text, stopwords_en)
    text = RemoveBlank(text)   
    text = TextToLower(text)
    
    return text

In [8]:
df['opinion'] = df['opinion'].apply(TextNormalizer)
df['title'] = df['title'].apply(TextNormalizer)
df['normal_text'] = df['title'].str.cat(df['opinion'], sep=' ')
df['translated'] = df['normal_text'].apply(TranslateDeepL) # Pode alternar entre métodos aqui, porém a lib é bem mais performática

display(df)

,score,title,opinion,normal_text,translated
81316,5,bom,gostei comprarei outras vezes,bom gostei comprarei outras vezes,good i liked it i'll buy more
47887,1,pessimo,produto com defeito,pessimo produto com defeito,bad product defective
16250,5,muito bom,deu tudo certo,muito bom deu tudo certo,very good everything went well
88016,5,excelente,entrega rapida produto de qualidade,excelente entrega rapida produto de qualidade,excellent fast delivery quality product
43520,5,super recomendo,produto muito util,super recomendo produto muito util,i highly recommend this product
99108,5,otimo,bom,otimo bom,great good
23989,5,recomendo,produto entregue dentro prazo e de acordo com ...,recomendo produto entregue dentro prazo e de a...,i recommend the product delivered on time and ...
46607,4,recomendo,produto bom,recomendo produto bom,I recommend a good product
78774,5,super recomndo,produto e lindo e de excelente qualidade,super recomndo produto e lindo e de excelente ...,"i highly recommend the product, it's beautiful..."
93284,4,chegou antes prazo,recomendo essa loja produto chegou antes previsto,chegou antes prazo recomendo essa loja produto...,arrived ahead of schedule i recommend this sto...


In [9]:
##### A partir daqui pode ser aplicado análise de sentimento e emoção no texto!

##### Análise de Emoção

In [10]:
# config da chave API
authenticator = IAMAuthenticator(ibm_key)

# Cria uma instância do serviço NLU e define a URL
natural_language_understanding = NaturalLanguageUnderstandingV1(
    version='2021-08-01',
    authenticator=authenticator
)
natural_language_understanding.set_service_url('https://api.us-east.natural-language-understanding.watson.cloud.ibm.com/instances/186a3145-a256-4653-945a-cb77e4542307')

In [21]:
# Função
def EmotionAnalysis(text):
    try:
        # Solicitação de análise de emoção
        response = natural_language_understanding.analyze(
            text=text,
            features=Features(emotion=EmotionOptions())
        ).get_result()

        # Extrair emoções
        emocoes = response['emotion']['document']['emotion']
        return max(emocoes, key=emocoes.get)
    except:
        return "insuficiente"

print(EmotionAnalysis("Hello, i'm bad and you"))

sadness


##### Análise de Sentimento

In [12]:
# Essa função funciona melhor com o texto em português, mas nada impede de fazer em inglês e ver os resultados
def SentimentAnalysis(text):
    endpoint = azure_endpoint
    key = azure_key

    text_analytics_client = TextAnalyticsClient(endpoint=endpoint, credential=AzureKeyCredential(key))

    result = text_analytics_client.analyze_sentiment([text], show_opinion_mining=True)
    
    sentiment = result[0]['sentiment']
   
    return sentiment 

print(SentimentAnalysis("Teste"))

neutral


##### Aplicando

In [13]:
df['emotion'] = df['translated'].apply(EmotionAnalysis)
df['sentiment'] = df['normal_text'].apply(SentimentAnalysis)

In [14]:
display(df)

,score,title,opinion,normal_text,translated,emotion,sentiment
81316,5,bom,gostei comprarei outras vezes,bom gostei comprarei outras vezes,good i liked it i'll buy more,insuficiente,positive
47887,1,pessimo,produto com defeito,pessimo produto com defeito,bad product defective,insuficiente,negative
16250,5,muito bom,deu tudo certo,muito bom deu tudo certo,very good everything went well,insuficiente,positive
88016,5,excelente,entrega rapida produto de qualidade,excelente entrega rapida produto de qualidade,excellent fast delivery quality product,insuficiente,positive
43520,5,super recomendo,produto muito util,super recomendo produto muito util,i highly recommend this product,insuficiente,positive
99108,5,otimo,bom,otimo bom,great good,insuficiente,positive
23989,5,recomendo,produto entregue dentro prazo e de acordo com ...,recomendo produto entregue dentro prazo e de a...,i recommend the product delivered on time and ...,insuficiente,positive
46607,4,recomendo,produto bom,recomendo produto bom,I recommend a good product,insuficiente,positive
78774,5,super recomndo,produto e lindo e de excelente qualidade,super recomndo produto e lindo e de excelente ...,"i highly recommend the product, it's beautiful...",insuficiente,positive
93284,4,chegou antes prazo,recomendo essa loja produto chegou antes previsto,chegou antes prazo recomendo essa loja produto...,arrived ahead of schedule i recommend this sto...,insuficiente,positive
